In [1]:
import logging
import time
from concurrent.futures import ThreadPoolExecutor

import requests as rq
import pandas as pd

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# лист категорий (блоков) для статей в Ленте
lenta_blocs_indexes = [1,   # Общество/Россия
                       3,   # Бывший СССР
                       4,   # Экономика
                       5,   # Наука и техника
                       8,   # Спорт
                       37,  # Силовые структуры
                       48,  # Туризм/Путешествия
                       87]  # Забота о себе

Ниже представлены вспомогательные классы: ParamsDictType для параметров поиска и абстрактный класс для парсеров NewsParser. Изначально хотел использовать несколько источников, но потом отказался от этой идеи.

In [4]:
from abc import ABC, abstractmethod
from typing import TypedDict, NotRequired


class ParamsDictType(TypedDict):
    size: int
    search_from: int
    bloc: NotRequired[int]
    query: NotRequired[str]


class NewsParser(ABC):
    @abstractmethod
    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        pass

    @abstractmethod
    def parse(self) -> pd.DataFrame:
        pass

    @abstractmethod
    def _get_page_data(self, url: str) -> pd.DataFrame:
        pass


In [5]:
class LentaParser(NewsParser):
    @staticmethod
    def _get_url(params: ParamsDictType) -> str:
        base_url = (
            f"https://lenta.ru/search/v2/process?size={params['size']}&from={params['search_from']}"
            f"&sort=2&domain=1&modified,format=yyyy-MM-dd&modified,from=2023-01-01&type=1"
        )

        if 'query' in params and params['query']:
            base_url += f"&query={params['query']}"
        if 'bloc' in params and params['bloc']:
            base_url += f"&bloc={params['bloc']}"

        return base_url

    def _get_page_data(self, url: str) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей по текущему url
        """
        try:
            time.sleep(2)
            response = rq.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            if 'matches' in data:
                return pd.DataFrame(data['matches'])

            logging.warning(f"Статьи не найдены: {url}")
            return pd.DataFrame()
        except Exception as e:
            logging.error(f"Ошибка при загрузке url {url}: {e}")
            return pd.DataFrame()

    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        """
        Парсит переданную категорию и возвращает pd.DataFrame
        """
        size = 250  # скачиванием батчами, чтобы не вылететь по таймауту
        search_from = 1
        articles_df = pd.DataFrame()

        while search_from < 20000:
            params = {
                'size': size,
                'search_from': search_from,
            }
            if query:
                params['query'] = query
            if bloc:
                params['bloc'] = bloc

            url = self._get_url(params)
            page_data = self._get_page_data(url)

            if page_data.empty:
                logging.info(f"Не найдено данных для категории {bloc}")
                break

            articles_df = pd.concat([articles_df, page_data], ignore_index=True)
            search_from += size

        logging.info(f"Для категории {bloc} найдено {articles_df.shape[0]} статей")
        return articles_df

    def parse(self) -> pd.DataFrame:
        """
        Парсит все переданные категории и возвращает обший pd.DataFrame
        """
        df = pd.DataFrame()

        with ThreadPoolExecutor() as executor:
            results = list(executor.map(self.parse_category, lenta_blocs_indexes))

        for result in results:
            df = pd.concat([df, result], ignore_index=True)

        # Тема "Строительство" парсится отдельно через query, так как такого отдельного блока тем на Ленте нет
        query_results = self.parse_category(query='Строительство')
        query_results['bloc'] = 6
        df = pd.concat([df, query_results], ignore_index=True)

        logging.info(f"Всего статей: {df.shape[0]}")
        return df

In [6]:
lenta_parser = LentaParser()
lenta_df = lenta_parser.parse()
lenta_df.shape

2024-12-23 08:45:19,140 - INFO - Не найдено данных для категории 8
2024-12-23 08:45:19,142 - INFO - Для категории 8 найдено 9999 статей
2024-12-23 08:45:20,732 - INFO - Не найдено данных для категории 3
2024-12-23 08:45:20,735 - INFO - Для категории 3 найдено 9999 статей
2024-12-23 08:45:23,727 - INFO - Не найдено данных для категории 48
2024-12-23 08:45:23,730 - INFO - Для категории 48 найдено 9999 статей
2024-12-23 08:45:23,911 - INFO - Не найдено данных для категории 37
2024-12-23 08:45:23,914 - INFO - Для категории 37 найдено 9999 статей
2024-12-23 08:45:24,119 - INFO - Не найдено данных для категории 87
2024-12-23 08:45:24,121 - INFO - Для категории 87 найдено 9132 статей
2024-12-23 08:45:31,615 - INFO - Не найдено данных для категории 1
2024-12-23 08:45:31,617 - INFO - Для категории 1 найдено 9999 статей
2024-12-23 08:45:33,674 - INFO - Не найдено данных для категории 5
2024-12-23 08:45:33,676 - INFO - Для категории 5 найдено 9999 статей
2024-12-23 08:45:35,168 - INFO - Не найден

(84929, 16)

In [7]:
lenta_df.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1754207,https://lenta.ru/news/2024/12/23/v-rossii-nach...,В России началась шестидневная рабочая неделя,1734931866,1734931867,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2024/12/23/08/202...,1734931866,Фото: Konstantin Kokoshkin / Global Look Press...,В России началась шестидневная рабочая неделя,"Фото: Konstantin Kokoshkin / Global Look ..., ..."
1,1754198,https://lenta.ru/news/2024/12/23/mladshiy-serz...,Младший сержант под обстрелом удержал участок ...,1734931020,1734931206,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2024/12/23/08/202...,1734931020,Фото: Алексей Коновалов / ТАСС Анастасия Шейки...,Младший сержант под обстрелом удержал участок ...,Фото: Алексей Коновалов / ТАСС Анастасия ... о...
2,1754194,https://lenta.ru/news/2024/12/23/minoborony-za...,Минобороны заявило о попытке удара ВСУ по России,1734929938,1734929939,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2024/12/23/07/202...,1734929938,Фото: Власов Сергей / Globallook Press Анастас...,Минобороны заявило о попытке удара ВСУ по России,Фото: Власов Сергей / Globallook Press ...) пы...
3,1754193,https://lenta.ru/news/2024/12/23/gruzovik/,Грузовик c водкой попал в ДТП и перевернулся н...,1734929305,1734929372,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2024/12/23/07/202...,1734929305,Фото: Sergey Elagin / Business Online / Global...,Грузовик c водкой попал в ДТП и перевернулся н...,Фото: Sergey Elagin / Business Online / ... с ...
4,1754188,https://lenta.ru/news/2024/12/23/propavshuyu-z...,Пропавшую жительницу Луганска нашли без призна...,1734928745,1734929026,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2024/12/23/07/202...,1734928745,Фото: Konstantin Kokoshkin / Global Look Press...,Пропавшую жительницу Луганска нашли без призна...,Фото: Konstantin Kokoshkin / Global Look ... к...


In [8]:
lenta_df['bloc'].value_counts(normalize=True)

bloc
1     0.117734
3     0.117734
4     0.117734
5     0.117734
8     0.117734
37    0.117734
48    0.117734
87    0.107525
6     0.068339
Name: proportion, dtype: float64

In [9]:
df = lenta_df[['bloc', 'text']] # оставляем только признак текст и таргет тему (блок)
df = df[df['text'].notnull() & (df['text'].str.split().str.len() > 5)] # удаляем короткие и отсутствующие тексты
df = df.drop_duplicates() # удаляем дубликаты

df.shape

(84929, 2)

In [10]:
# Маппим индексы тем с сайта Ленты с названиями
lenta_categories = {
    1: 'Общество/Россия',
    3: 'Бывший СССР',
    4: 'Экономика',
    5: 'Наука и техника',
    8: 'Спорт',
    37: 'Силовые структуры',
    48: 'Туризм/Путешествия',
    87: 'Забота о себе',
    6: 'Строительство'
}

# Маппим индексы тем из задания с названиями
article_categories = {
    0: 'Общество/Россия',
    1: 'Экономика',
    2: 'Силовые структуры',
    3: 'Бывший СССР',
    4: 'Спорт',
    5: 'Забота о себе',
    6: 'Строительство',
    7: 'Туризм/Путешествия',
    8: 'Наука и техника'
}

new_mapping = {v: k for k, v in article_categories.items()}

# Подменяем индексы тем на нужные по заданию
df['bloc'] = df['bloc'].dropna().map(lenta_categories).map(new_mapping)
df['bloc'].value_counts(normalize=True)

bloc
0    0.117734
3    0.117734
1    0.117734
8    0.117734
4    0.117734
2    0.117734
7    0.117734
5    0.107525
6    0.068339
Name: proportion, dtype: float64

In [11]:
df.to_csv('./lenta_df.csv')